In [43]:
prompt_text="The crux of calvinism is its doctrine of the soverignty of God. If God is soverign then he controls everything. Thus the doctrine of predestination holds"
#max reponse_length 1024
response_length=1000
#output_file will be created if it doesn't exist, otherwise answers will be appended
output_file="results.csv"
#max 4 (k80 gpu)
num_of_responses=4


In [44]:
!nvidia-smi

Fri Apr 10 18:57:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.64       Driver Version: 430.64       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    60W / 149W |  11430MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [45]:
all_seq=[]

In [46]:
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

#device = xm.xla_device()
device=torch.device("cuda")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
model.to(device)
encoded_prompt=tokenizer.encode(prompt_text, add_special_tokens=True,return_tensors="pt")
encoded_prompt = encoded_prompt.to(device)

outputs = model.generate(encoded_prompt,response_length,temperature=.8,num_return_sequences=num_of_responses,repetition_penalty=85,do_sample=True,top_k=80,top_p=.85 )


RuntimeError: CUDA out of memory. Tried to allocate 308.00 MiB (GPU 0; 11.17 GiB total capacity; 10.58 GiB already allocated; 10.81 MiB free; 10.85 GiB reserved in total by PyTorch)

In [ ]:
# Remove the batch dimension when returning multiple sequences
if len(outputs.shape) > 2:
    outputs.squeeze_()

In [ ]:
generated_sequences=[]
total_sequence=""
for generated_sequence_idx, generated_sequence in enumerate(outputs):
        print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()
        #print(generated_sequence)
        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        #print(text)
        # Remove all text after the stop token
        stop_token='<|endoftext|>'
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
        print(total_sequence)
        all_seq.append(total_sequence)

In [ ]:
if len(all_seq)==len(set(all_seq)):
  print('no duplicates')
else:
  print('duplicates found')

In [ ]:
for i in all_seq:
  print(i)

In [ ]:
import csv
import os
if os.path.exists(output_file):
    append_flag="a"
else: 
    append_flag="w"
with open (output_file, append_flag) as csvfile:
    writer=csv.writer(csvfile)
    for i in all_seq:
        writer.writerow([i])
    